# Create Vacancy Jobs

# Post Vacancy

In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

from ase.io import read,write
from ase.mep import DyNEB
from mace.calculators.mace import MACECalculator

In [7]:
# read in the start and end structures 
#start_structure = read('./scratch/data/vasp_jobs-neb-test/vac/Cr5Ti9V107W4Zr3/Start_Index_34/OUTCAR')
#end_structure = read('./scratch/data/vasp_jobs-neb-test/vac/Cr5Ti9V107W4Zr3/End_Index_110/OUTCAR')
start_structure = read('./scratch/data/vasp_jobs-neb-test/vac/Cr4Ti7V111W4Zr2/Start_Index_14/OUTCAR')
end_structure = read('./scratch/data/vasp_jobs-neb-test/vac/Cr4Ti7V111W4Zr2/End_Index_15/OUTCAR')

In [8]:
images = [start_structure.copy() for i in range(6)]
images.append(end_structure.copy())

print(images)
print(len(images))

[Atoms(symbols='Cr4Ti7V110W4Zr2', pbc=True, cell=[[12.093483239, -0.001965245, -0.001402732], [-0.001967216, 12.088895931, 0.017572922], [-0.001397384, 0.01757218, 12.092468779]]), Atoms(symbols='Cr4Ti7V110W4Zr2', pbc=True, cell=[[12.093483239, -0.001965245, -0.001402732], [-0.001967216, 12.088895931, 0.017572922], [-0.001397384, 0.01757218, 12.092468779]]), Atoms(symbols='Cr4Ti7V110W4Zr2', pbc=True, cell=[[12.093483239, -0.001965245, -0.001402732], [-0.001967216, 12.088895931, 0.017572922], [-0.001397384, 0.01757218, 12.092468779]]), Atoms(symbols='Cr4Ti7V110W4Zr2', pbc=True, cell=[[12.093483239, -0.001965245, -0.001402732], [-0.001967216, 12.088895931, 0.017572922], [-0.001397384, 0.01757218, 12.092468779]]), Atoms(symbols='Cr4Ti7V110W4Zr2', pbc=True, cell=[[12.093483239, -0.001965245, -0.001402732], [-0.001967216, 12.088895931, 0.017572922], [-0.001397384, 0.01757218, 12.092468779]]), Atoms(symbols='Cr4Ti7V110W4Zr2', pbc=True, cell=[[12.093483239, -0.001965245, -0.001402732], [-0.00

In [9]:
for image in images:
    image.calc = MACECalculator(model_paths=["./forge/tests/resources/potentials/mace/gen_5_model_0-11-28_stagetwo.model"], device='cuda',enable_cueq=True, default_dtype='float32')


Converting models to CuEq for acceleration
Converting models to CuEq for acceleration
Converting models to CuEq for acceleration
Converting models to CuEq for acceleration
Converting models to CuEq for acceleration
Converting models to CuEq for acceleration
Converting models to CuEq for acceleration


In [10]:
neb = DyNEB(images)
neb.interpolate(mic=True)

In [11]:
write('./scratch/data/vasp_jobs-neb-test/vac/Cr4Ti7V111W4Zr2/neb_images.xyz', images)

### Psuedo code for making the vasp neb job

In [ ]:
from ase.io import read,write
from ase.mep import DyNEB

start_structure = read('./scratch/data/vasp_jobs-neb-test/vac/Cr4Ti7V111W4Zr2/Start_Index_14/OUTCAR')
end_structure = read('./scratch/data/vasp_jobs-neb-test/vac/Cr4Ti7V111W4Zr2/End_Index_15/OUTCAR')

images = [start_structure.copy() for i in range(6)]
images.append(end_structure.copy())

neb = DyNEB(images)
neb.interpolate(mic=True)

# make vasp neb job from the images
# from 00 to 0X where X = len(images) - 1, put each image in the correct directory, i.e images[0] in 00, images[1] in 01, etc. 
# copy the OUTCAR from the start and end structures to the 00 and 0X directories
# use the neb-vtst or neb.json file to create the INCAR, KPOINTS, and POTCAR files 
# use an hpc profile (for now Perlmutter-GPU-NEB.json) to create the slurm script in that folder

# Using new prepare_neb_vasp_job function


In [3]:
import os
from forge.workflows.db_to_vasp import prepare_neb_vasp_job

os.environ['VASP_PP_PATH'] = '/home/myless/Packages/VASP/POTCAR_64_PBE'
start_path = '/home/myless/Packages/forge/scratch/data/vasp_jobs-neb-test/vac/Cr4Ti7V111W4Zr2/Start_Index_14/OUTCAR'
end_path = '/home/myless/Packages/forge/scratch/data/vasp_jobs-neb-test/vac/Cr4Ti7V111W4Zr2/End_Index_15/OUTCAR'
output_dir = '/home/myless/Packages/forge/scratch/data/vasp_jobs-neb-test/neb/Cr4Ti7V111W4Zr2_14_to_15/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

prepare_neb_vasp_job(start_outcar=start_path,
                    end_outcar=end_path,
                    n_images=5,
                    vasp_profile_name='neb-vtst',
                    hpc_profile_name='Perlmutter-GPU-NEB',
                    output_dir= output_dir,
                    job_name='Cr4Ti7V111W4Zr2_14_to_15')

[INFO] Created NEB job in /home/myless/Packages/forge/scratch/data/vasp_jobs-neb-test/neb/Cr4Ti7V111W4Zr2_14_to_15/ using HPC=Perlmutter-GPU-NEB, VASP=neb-vtst
[INFO] NEB path visualization saved to /home/myless/Packages/forge/scratch/data/vasp_jobs-neb-test/neb/Cr4Ti7V111W4Zr2_14_to_15/neb_path.xyz


# Testing new defects

In [9]:
from dataclasses import dataclass
from typing import List, Tuple, Optional
import numpy as np
from ase import Atoms, Atom
from ase.build import bulk, surface
from ase.visualize import view
import numpy.linalg as la

@dataclass
class BCCDefectParams:
    a0: float     # lattice parameter
    size: Tuple[int, int, int]
    vacuum: float = 0.0
    pbc: Tuple[bool, bool, bool] = (True, True, True)

class BCCInterstitialGenerator:
    def __init__(self, params: BCCDefectParams):
        self.params = params
        self.base_structure = bulk('V', 'bcc', a=params.a0).repeat(params.size)
        
    def create_tetrahedral(self, position: str = 'center') -> Atoms:
        """Create tetrahedral interstitial"""
        structure = self.base_structure.copy()
        if position == 'center':
            cell_center = np.sum(structure.cell, axis=0) / 2
            # Tetrahedral position at (1/4, 1/4, 1/4)a0
            tet_pos = cell_center + np.array([0.25, 0.25, 0.25]) * self.params.a0
            structure.append(Atom('Ti', tet_pos))
        return structure
    
    def create_octahedral(self, position: str = 'center') -> Atoms:
        """Create octahedral interstitial"""
        structure = self.base_structure.copy()
        if position == 'center':
            cell_center = np.sum(structure.cell, axis=0) / 2
            # Octahedral position at (1/2, 1/2, 1/2)a0
            oct_pos = cell_center + np.array([0.5, 0.5, 0.5]) * self.params.a0
            structure.append(Atom('Ti', oct_pos))
        return structure
    
    def create_crowdion(self, direction: str = '111') -> Atoms:
        """Create crowdion (3+ atoms in a row)"""
        structure = self.base_structure.copy()
        center = len(structure) // 2
        
        if direction == '111':
            disp = np.array([1, 1, 1]) / np.sqrt(3)
        elif direction == '110':
            disp = np.array([1, 1, 0]) / np.sqrt(2)
        
        positions = structure.get_positions()
        
        # Find atoms in the crowdion direction
        center_pos = positions[center]
        distances = np.array([np.dot(pos - center_pos, disp) for pos in positions])
        crowdion_atoms = np.where(np.abs(distances) < 2 * self.params.a0)[0]
        
        # Displace 3+ atoms
        for idx in crowdion_atoms:
            dist = distances[idx]
            if abs(dist) < 1.5 * self.params.a0:
                positions[idx] += 0.15 * self.params.a0 * np.sign(dist) * disp
                structure[idx].symbol = 'Ti'  # Change atom type to Ti for debugging
                
        structure.set_positions(positions)
        return structure

class BCCDislocationGenerator:
    def __init__(self, params: BCCDefectParams):
        self.params = params
        
    def create_edge_dislocation(self, plane: str = '110') -> Atoms:
        """Create edge dislocation"""
        # Create larger system for dislocation
        size = (20, 20, 4)  # Larger cross-section needed
        structure = bulk('V', 'bcc', a=self.params.a0, cubic=True).repeat(size)
        
        # Burgers vector b = 1/2[111]
        b = self.params.a0 * np.array([1, 1, 1]) / 2
        
        # Calculate displacement field
        positions = structure.get_positions()
        center = np.mean(positions, axis=0)
        
        for i, pos in enumerate(positions):
            x = pos[0] - center[0]
            y = pos[1] - center[1]
            theta = np.arctan2(y, x)
            r = np.sqrt(x**2 + y**2)
            
            if r > 0.1:  # Avoid singularity at core
                ux = (b[0]/(2*np.pi)) * (theta + (x*y)/(2*(1-0.3)*r**2))
                uy = -(b[0]/(2*np.pi)) * ((1-2*0.3)*(np.log(r/self.params.a0))/4 + (x**2-y**2)/(4*(1-0.3)*r**2))
                positions[i][0] += ux
                positions[i][1] += uy
                
        structure.set_positions(positions)
        return structure
    
    def create_screw_dislocation(self) -> Atoms:
        """Create screw dislocation with b = 1/2[111]"""
        size = (20, 20, 4)
        structure = bulk('V', 'bcc', a=self.params.a0).repeat(size)
        
        # Burgers vector b = 1/2[111]
        b = self.params.a0 * np.array([1, 1, 1]) / 2
        
        positions = structure.get_positions()
        center = np.mean(positions, axis=0)
        
        for i, pos in enumerate(positions):
            x = pos[0] - center[0]
            y = pos[1] - center[1]
            theta = np.arctan2(y, x)
            
            # Displacement only in z direction for screw dislocation
            uz = (b[2]/(2*np.pi)) * theta
            positions[i][2] += uz
            
        structure.set_positions(positions)
        return structure



In [11]:
# Example usage:
def create_test_structures():
    params = BCCDefectParams(a0=3.03, size=(4, 4, 4))  # V lattice parameter
    
    int_gen = BCCInterstitialGenerator(params)
    dis_gen = BCCDislocationGenerator(params)
    
    # Generate structures
    tet = int_gen.create_tetrahedral()
    oct = int_gen.create_octahedral()
    crowdion = int_gen.create_crowdion('111')
    edge = dis_gen.create_edge_dislocation()
    screw = dis_gen.create_screw_dislocation()
    
    return {
        'tetrahedral': tet,
        'octahedral': oct,
        'crowdion': crowdion,
        'edge_dislocation': edge,
        'screw_dislocation': screw
    }

create_test_structures()

{'tetrahedral': Atoms(symbols='TiV64', pbc=True, cell=[[-6.06, 6.06, 6.06], [6.06, -6.06, 6.06], [6.06, 6.06, -6.06]]),
 'octahedral': Atoms(symbols='TiV64', pbc=True, cell=[[-6.06, 6.06, 6.06], [6.06, -6.06, 6.06], [6.06, 6.06, -6.06]]),
 'crowdion': Atoms(symbols='Ti60V4', pbc=True, cell=[[-6.06, 6.06, 6.06], [6.06, -6.06, 6.06], [6.06, 6.06, -6.06]]),
 'edge_dislocation': Atoms(symbols='V3200', pbc=True, cell=[60.599999999999994, 60.599999999999994, 12.12]),
 'screw_dislocation': Atoms(symbols='V1600', pbc=True, cell=[[-30.299999999999997, 30.299999999999997, 30.299999999999997], [30.299999999999997, -30.299999999999997, 30.299999999999997], [6.06, 6.06, -6.06]])}

In [12]:
import os
from ase.io import write,read
save_dir = './scratch/data/defects'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for key, value in create_test_structures().items():
    write(os.path.join(save_dir,f'{key}.xyz'), value)

In [1]:
from dataclasses import dataclass
from typing import List, Tuple, Optional
import numpy as np
from ase import Atoms, Atom
from ase.build import bulk
import numpy.linalg as la

@dataclass
class BCCDefectParams:
    a0: float     # lattice parameter
    size: Tuple[int, int, int]
    vacuum: float = 0.0
    pbc: Tuple[bool, bool, bool] = (True, True, True)

class BCCInterstitialGenerator:
    def __init__(self, params: BCCDefectParams):
        self.params = params
        self.base_structure = bulk('V', 'bcc', a=params.a0).repeat(params.size)
        
    def create_tetrahedral(self, position: str = 'center') -> Atoms:
        """Create tetrahedral interstitial at (1/4, 1/4, 1/4) relative to a unit cell."""
        structure = self.base_structure.copy()
        if position == 'center':
            # Find the central unit cell in the supercell
            central_cell = [s//2 for s in self.params.size]
            # Tetrahedral position at (1/4, 1/4, 1/4) of the central unit cell
            tet_pos = (
                (central_cell[0] + 0.25) * self.params.a0,
                (central_cell[1] + 0.25) * self.params.a0,
                (central_cell[2] + 0.25) * self.params.a0
            )
            structure.append(Atom('V', tet_pos))
        return structure
    
    def create_octahedral(self, position: str = 'center') -> Atoms:
        """Create octahedral interstitial at a face center position."""
        structure = self.base_structure.copy()
        if position == 'center':
            # Find the central unit cell in the supercell
            central_cell = [s//2 for s in self.params.size]
            # Octahedral position at face center (1/2, 1/2, 0) of the central unit cell
            oct_pos = (
                (central_cell[0] + 0.5) * self.params.a0,
                (central_cell[1] + 0.5) * self.params.a0,
                central_cell[2] * self.params.a0
            )
            # Verify no overlap
            distances = np.linalg.norm(structure.get_positions() - oct_pos, axis=1)
            if np.min(distances) < 0.5 * self.params.a0:
                raise ValueError("Octahedral site overlaps with existing atoms.")
            structure.append(Atom('V', oct_pos))
        return structure
    
    def create_crowdion(self, direction: str = '111') -> Atoms:
        """Create crowdion by displacing atoms along a close-packed direction."""
        structure = self.base_structure.copy()
        positions = structure.get_positions()
        center = np.mean(positions, axis=0)
        
        if direction == '111':
            disp = np.array([1, 1, 1], dtype=float)
        elif direction == '110':
            disp = np.array([1, 1, 0], dtype=float)
        else:
            raise ValueError("Direction must be '111' or '110'.")
        
        disp /= la.norm(disp)  # Normalize direction vector
        
        # Find atoms along the specified direction near the center
        line_data = []
        for i, pos in enumerate(positions):
            vec = pos - center
            parallel_dist = np.dot(vec, disp)
            perpendicular = vec - parallel_dist * disp
            line_data.append((parallel_dist, la.norm(perpendicular), i))
        
        # Select atoms within 0.2*a0 of the line
        threshold = 0.2 * self.params.a0
        selected = [item for item in line_data if item[1] < threshold]
        selected.sort(key=lambda x: x[0])  # Sort by position along the direction
        
        # Select 3-5 atoms in the middle to displace
        if len(selected) < 3:
            raise RuntimeError("Not enough atoms along the crowdion direction.")
        
        mid = len(selected) // 2
        crowdion_indices = [item[2] for item in selected[mid-1:mid+2]]
        
        # Displace atoms along the direction
        displacement = 0.3 * self.params.a0 * disp
        for idx in crowdion_indices:
            positions[idx] += displacement
        
        structure.set_positions(positions)
        return structure

# Example usage:
def create_test_structures():
    params = BCCDefectParams(a0=3.03, size=(3, 3, 3))  # 3x3x3 supercell for clarity
    
    int_gen = BCCInterstitialGenerator(params)
    
    # Generate structures
    tet = int_gen.create_tetrahedral()
    oct = int_gen.create_octahedral()
    crowdion = int_gen.create_crowdion('111')
    
    return {
        'tetrahedral': tet,
        'octahedral': oct,
        'crowdion': crowdion
    }

In [2]:
create_test_structures()
import os
from ase.io import write,read
save_dir = './scratch/data/new_defects'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for key, value in create_test_structures().items():
    write(os.path.join(save_dir,f'{key}.xyz'), value)

In [3]:
from dataclasses import dataclass
from typing import Tuple
import numpy as np
from ase import Atoms, Atom
from ase.build import bulk
import numpy.linalg as la

@dataclass
class BCCDefectParams:
    a0: float     # lattice parameter
    size: Tuple[int, int, int]
    vacuum: float = 10.0  # Add vacuum for visualization
    pbc: Tuple[bool, bool, bool] = (False, False, False)  # Turn off PBC for visualization

class BCCDefectGenerator:
    def __init__(self, params: BCCDefectParams):
        self.params = params
        self.base_structure = self._create_base_structure()
        
    def _create_base_structure(self) -> Atoms:
        """Create base structure with vacuum and centered atoms"""
        bulk_struct = bulk('V', 'bcc', a=self.params.a0).repeat(self.params.size)
        bulk_struct.center(vacuum=self.params.vacuum)
        return bulk_struct

    def create_tetrahedral(self) -> Atoms:
        """Create tetrahedral interstitial with hydrogen marker"""
        structure = self.base_structure.copy()
        cell_center = np.diag(structure.cell)/2
        tet_pos = cell_center + 0.25*self.params.a0*np.array([1,1,1])
        
        # Add small hydrogen atom for visualization
        structure.append(Atom('H', tet_pos))
        return structure

    def create_octahedral(self) -> Atoms:
        """Create octahedral interstitial with hydrogen marker"""
        structure = self.base_structure.copy()
        cell_center = np.diag(structure.cell)/2
        
        # Octahedral site at face center
        oct_pos = cell_center + 0.5*self.params.a0*np.array([1,1,0])
        
        # Add hydrogen marker
        structure.append(Atom('H', oct_pos))
        return structure

    def create_crowdion(self, direction: str = '111') -> Atoms:
        """Create crowdion with titanium markers"""
        structure = self.base_structure.copy()
        positions = structure.get_positions()
        center = np.diag(structure.cell)/2
        
        # Get direction vector
        if direction == '111':
            disp_dir = np.array([1,1,1])
        elif direction == '110':
            disp_dir = np.array([1,1,0])
        disp_dir = disp_dir/la.norm(disp_dir)
        
        # Find atoms along the crowdion line
        line_atoms = []
        for i, pos in enumerate(positions):
            vec = pos - center
            projection = np.dot(vec, disp_dir)
            if abs(projection) < 2*self.params.a0:  # Select along direction
                line_atoms.append((projection, i))
        
        # Sort and select 5 central atoms
        line_atoms.sort(key=lambda x: x[0])
        crowdion_indices = [x[1] for x in line_atoms[len(line_atoms)//2-2:len(line_atoms)//2+3]]
        
        # Displace atoms and change to Ti
        displacement = 0.4*self.params.a0*disp_dir
        for idx in crowdion_indices:
            structure[idx].position += displacement
            structure[idx].symbol = 'Ti'  # Change element for visualization
            
        return structure

def visualize_defects(structures: dict):
    """Helper function for visualization analysis"""
    from ase.visualize import view
    for name, struct in structures.items():
        print(f"\n--- {name.upper()} DEFECT ANALYSIS ---")
        print("Use these visualization tips:")
        print("1. Rotate to see central region")
        print("2. In Ovito: Color by 'Particle Type' (H=white, Ti=blue)")
        print("3. Enable coordinate axes (Ctrl+A)")
        print("4. Check nearest neighbor distances")
        view(struct, viewer='ovito')

In [8]:
params = BCCDefectParams(
    a0=3.03,
    size=(5,5,5),  # Larger size for clearer visualization
    vacuum=10.0,    # Adds empty space around structure
    pbc=(False, False, False)  # Disable periodic boundaries for visualization
)


In [11]:
gen = BCCDefectGenerator(params)

# Usage:
defects = {
    'tetrahedral': gen.create_tetrahedral(),
    'octahedral': gen.create_octahedral(),
    'crowdion': gen.create_crowdion('111')
}
visualize_defects(defects)



--- TETRAHEDRAL DEFECT ANALYSIS ---
Use these visualization tips:
1. Rotate to see central region
2. In Ovito: Color by 'Particle Type' (H=white, Ti=blue)
3. Enable coordinate axes (Ctrl+A)
4. Check nearest neighbor distances


KeyError: 'ovito'

In [13]:
import os
from ase.io import write,read
save_dir = './scratch/data/new_defects'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for key, value in create_test_structures().items():
    write(os.path.join(save_dir,f'{key}.xyz'), value)

# Making static vasp jobs for finished NEB jobs 

In [3]:
import os
from ase.io import read
from forge.workflows.db_to_vasp import prepare_vasp_job_from_ase

os.environ['VASP_PP_PATH'] = '/home/myless/Packages/VASP/POTCAR_64_PBE'

main_data_path = './scratch/data/mrs_neb/'
main_output_dir = './scratch/data/mrs_neb_static/'
num_images = 5

# get the folders in the main_data_path
folders = [f for f in os.listdir(main_data_path) if os.path.isdir(os.path.join(main_data_path, f))]

for folder in folders:
    for i in range(num_images+2):
        print(f'Processing {folder} image {i}')
        atoms = read(os.path.join(main_data_path, folder, f'0{i}/OUTCAR'))
        job_output_dir = os.path.join(main_output_dir, folder, f'0{i}')
        prepare_vasp_job_from_ase(atoms, vasp_profile_name='static', hpc_profile_name='PSFC-GPU', output_dir=job_output_dir, auto_kpoints=True, DEBUG=True, job_name=str(folder)+f'_0{i}')

Processing Cr6Ti11V102W6Zr3_60_to_54 image 0
[DEBUG] Looking for HPC profile in: /home/myless/Packages/forge/forge/workflows/hpc_profiles
[DEBUG] Looking for VASP profile in: /home/myless/Packages/forge/forge/workflows/vasp_settings
[DEBUG] HPC profile directory exists: True
[DEBUG] VASP profile directory exists: True
[DEBUG] Available HPC profiles: [PosixPath('/home/myless/Packages/forge/forge/workflows/hpc_profiles/Perlmutter-CPU.json'), PosixPath('/home/myless/Packages/forge/forge/workflows/hpc_profiles/PSFC-GPU.json'), PosixPath('/home/myless/Packages/forge/forge/workflows/hpc_profiles/Perlmutter-GPU-NEB.json')]
[DEBUG] Available VASP profiles: [PosixPath('/home/myless/Packages/forge/forge/workflows/vasp_settings/neb.json'), PosixPath('/home/myless/Packages/forge/forge/workflows/vasp_settings/static.json'), PosixPath('/home/myless/Packages/forge/forge/workflows/vasp_settings/relaxation.json'), PosixPath('/home/myless/Packages/forge/forge/workflows/vasp_settings/neb-vtst.json')]

[D